In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.regression import LinearRegression


spark = SparkSession.builder.getOrCreate()
spark

In [2]:
data_path = '/Users/udhhav/Desktop/pyspark/LinkedIn/'
file_path = data_path+"utlization.json"

In [3]:
df_util = spark.read.format("json").load(file_path)

In [4]:
df_util.createOrReplaceTempView('utilization')
df_util.count()

500000

In [5]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [6]:
vector_assembler = VectorAssembler(inputCols=["cpu_utilization","free_memory","session_count"],outputCol="features")

In [8]:
vcluster_df = vector_assembler.transform(df_util)

In [9]:
vcluster_df.show()

+---------------+-------------------+-----------+---------+-------------+----------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|
+---------------+-------------------+-----------+---------+-------------+----------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.35,58.0]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58| [0.41,0.4,58.0]|

In [10]:
kmeans = KMeans().setK(3)

In [11]:
kmeans = kmeans.setSeed(1)

In [13]:
kmodel = kmeans.fit(vcluster_df)

In [14]:
kmodel .clusterCenters()

[array([ 0.52047775,  0.47836303, 51.79927162]),
 array([ 0.62881549,  0.37094643, 70.43030159]),
 array([ 0.71931575,  0.28104316, 88.23965784])]

In [16]:
vectorAssembler = VectorAssembler(inputCols=["cpu_utilization"],outputCol="features")

In [17]:
df_vutil = vectorAssembler.transform(df_util)
df_vutil.show()

+---------------+-------------------+-----------+---------+-------------+--------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|features|
+---------------+-------------------+-----------+---------+-------------+--------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|  [0.57]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|  [0.47]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|  [0.56]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|  [0.57]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|  [0.35]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|  [0.41]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|  [0.57]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|  [0.41]|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|  [0.53]|
|   

In [18]:
lr = LinearRegression(featuresCol="features",labelCol="session_count")

In [19]:
lrModel = lr.fit(df_vutil)

22/05/16 22:05:01 WARN Instrumentation: [5734af0c] regParam is zero, which might cause numerical instability and overfitting.
22/05/16 22:05:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/05/16 22:05:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/05/16 22:05:03 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [21]:
lrModel.coefficients

DenseVector([47.024])

In [22]:
lrModel.intercept

40.41695103556804

In [23]:
lrModel.summary.rootMeanSquaredError

12.837990225931815